# Calculating Distance Lab

### Introduction

In this lab, you will write methods to calculate the distance of various students in a class from each other.  Once again, let's assume that the $x$ coordinates represent avenues of a student, the $y$ coordinates represent street numbers, and we will assume that the distance between each street and the distance between each avenue is the same.

In this lab we will write functions to calculate the distance between students.  We will work up to a function called `nearest_neighbors` that given a student, finds the other students who are closest.

### Getting Started

Let's declare a variable `students` and assign it to an array of dictionaries, each representing the location of a student.

In [4]:
neighbors = [{'name': 'Bob', 'avenue': 4, 'street': 8}, {'name': 'Suzie', 'avenue': 1, 'street': 11}, 
             {'name': 'Fred', 'avenue': 5, 'street': 8}, {'name': 'Edgar', 'avenue': 6, 'street': 13},
             {'name': 'Steven', 'avenue': 3, 'street': 6}, {'name': 'Natalie', 'avenue': 5, 'street': 4}]

In [5]:
neighbors

[{'avenue': 4, 'name': 'Bob', 'street': 8},
 {'avenue': 1, 'name': 'Suzie', 'street': 11},
 {'avenue': 5, 'name': 'Fred', 'street': 8},
 {'avenue': 6, 'name': 'Edgar', 'street': 13},
 {'avenue': 3, 'name': 'Steven', 'street': 6},
 {'avenue': 5, 'name': 'Natalie', 'street': 4}]

In [11]:
fred = neighbors[0]
natalie = neighbors[5]

We'll also plot our students, to get a sense of our data.

In [12]:
import plotly

plotly.offline.init_notebook_mode(connected=True)
trace0 = dict(x=list(map(lambda neighbor: neighbor['avenue'],neighbors)), 
              y=list(map(lambda neighbor: neighbor['street'],neighbors)),
              text=list(map(lambda neighbor: neighbor['name'],neighbors)),
              mode='markers')
plotly.offline.iplot(dict(data=[trace0], layout={'xaxis': {'dtick': 1}, 'yaxis': {'dtick': 1}}))

We'll start by focusing on the customers Fred and Natalie, and points (4, 8) and (5, 4) respectively.

### Calculating the sides of the triangle

Remember that to calculate the distance, we use the Pythagorean Theorem to calculate the two shorter sides of the right triangle, and from there can calculate the distance, that is the hypotenuse, of the triangle.  Let's start with calculating the shorter sides and then use that work to calculate the distance. 

Write a function called `street_distance` that calculates how far **in streets** two students are from each other.  So for example, with Daniel at street 1, and Fred at street 4, it should return 3.

In [13]:
def street_distance(first_neighbor, second_neighbor):
        return first_neighbor['street'] - second_neighbor['street']

Now we execute the code, and as you can see from the comment to the right, our expected returned street distance is $3$.

In [15]:
street_distance(fred, natalie) # 4

4

Write a function called `avenue_distance` that calculates how far in avenues two students are from each other.

In [16]:
def avenue_distance(first_neighbor, second_neighbor):
    return abs(first_neighbor['avenue'] - second_neighbor['avenue'])

In [17]:
avenue_distance(fred, natalie) #  == 3

1

### Calculating the distance

Now let's begin writing functions involved with calculating that hypotenuse of our right triangle.  Using the Pythagorean Theorem, write a function called `distance_between_students_squared` that calculates the length of the hypotenuse squared.

In [18]:
def distance_between_students_squared(first_neighbor, second_neighbor):
    return street_distance(first_neighbor, second_neighbor)**2 + avenue_distance(first_neighbor, second_neighbor)**2

In [19]:
distance_between_students_squared(fred, daniel) # 10

18

Now take the next step, and write a function called `distance`, that given two students returns the distance between them.  

In [20]:
import math
def distance(first_neighbor, second_neighbor):
    return math.sqrt(distance_between_students_squared(first_neighbor, second_neighbor))

In [22]:
distance(fred, natalie)

4.123105625617661

### Writing Our "Nearest Neighbors" Functions

This next section will work up to building a `nearest_neighbor` function.  This is a function that given one student, will tell us which students are closest.  How do we write something like this? Can we use our calculation of distance between two students, to figure out the closest students to an individual?

Sure, we first need to calculate the distances between one student and all of the others.  Next, we sort those students by their distance from the student.  Finally, we select a given number of the closest students.  Let's work through it.   

Note that we already have a function that calculates the distance between two students.  We may think we could simply use this function to loop through our students, but that would just return an array of distances.  

In [23]:
distances = []
for neighbor in neighbors:
    distance_between = distance(fred, neighbor)
    distances.append(distance_between)

distances

[0.0,
 4.242640687119285,
 1.0,
 5.385164807134504,
 2.23606797749979,
 4.123105625617661]

The returned array from the above procedure isn't super helpful.  We need to know who each distance is associated with.  

So let's accomplish this by writing a function called `distance_with_student` that works like our distance function but instead of returning an integer, returns a dictionary representing the `second_student`, and also adds in the a key value pair indicating distance from the `first_student`.

In [24]:
import math
def distance_with_student(first_neighbor, second_neighbor):
    student_with_distance = second_neighbor.copy()
    distance = math.sqrt(distance_between_students_squared(first_neighbor, second_neighbor))
    student_with_distance['distance'] = distance
    return student_with_distance

In [25]:
distance_with_student(fred, natalie)
# {'avenue': 5, 'distance': 4.123105625617661, 'name': 'Natalie', 'street': 4}

{'avenue': 5, 'distance': 4.123105625617661, 'name': 'Natalie', 'street': 4}

Now write a function called `distance_all` that returns an array representing the distances between a `first_student` and the rest of the students.  The array should not return the `first_student` in its collection of students. 

In [26]:
def distance_all(first_neighbor, neighbors):
    remaining_neighbors = filter(lambda neighhbor: neighbor != first_neighbor, neighbors)
    return list(map(lambda neighbor: distance_with_student(first_neighbor, neighbor), remaining_neighbors))

In [28]:
distance_all(fred, neighbors)

# [{'avenue': 4, 'distance': 0.0, 'name': 'Bob', 'street': 8},
#  {'avenue': 1, 'distance': 4.242640687119285, 'name': 'Suzie', 'street': 11},
#  {'avenue': 5, 'distance': 1.0, 'name': 'Fred', 'street': 8},
#  {'avenue': 6, 'distance': 5.385164807134504, 'name': 'Edgar', 'street': 13},
#  {'avenue': 3, 'distance': 2.23606797749979, 'name': 'Steven', 'street': 6},
#  {'avenue': 5, 'distance': 4.123105625617661, 'name': 'Natalie', 'street': 4}]

[{'avenue': 4, 'distance': 0.0, 'name': 'Bob', 'street': 8},
 {'avenue': 1, 'distance': 4.242640687119285, 'name': 'Suzie', 'street': 11},
 {'avenue': 5, 'distance': 1.0, 'name': 'Fred', 'street': 8},
 {'avenue': 6, 'distance': 5.385164807134504, 'name': 'Edgar', 'street': 13},
 {'avenue': 3, 'distance': 2.23606797749979, 'name': 'Steven', 'street': 6},
 {'avenue': 5, 'distance': 4.123105625617661, 'name': 'Natalie', 'street': 4}]

Finally, write a function called `nearest_neighbors` given a student, returns an array of students, ordered from closest to furthest from the student.  The function should take an optional third argument that specifies how many "nearest" students are returned.

In [29]:
def nearest_neighbors(first_neighbor, neighbors, number = None):
    number = number or len(neighbors) - 1
    neighbor_distances = distance_all(first_neighbor, neighbors)
    sorted_neighbors = sorted(neighbor_distances, key=lambda neighbor: neighbor['distance'])
    return sorted_neighbors[:number]

In [31]:
nearest_neighbors(fred, neighbors, 2)
# [{'avenue': 10, 'distance': 1.4142135623730951, 'name': 'steven', 'street': 4},
#  {'avenue': 5, 'distance': 2.0, 'name': 'daniel', 'street': 1}]

[{'avenue': 4, 'distance': 0.0, 'name': 'Bob', 'street': 8},
 {'avenue': 5, 'distance': 1.0, 'name': 'Fred', 'street': 8}]

### Summary

In this lab, you built out the nearest neighbors.  We'll review building out these functions in the next section.